In [66]:
from pathlib import Path
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import datetime
import re

In [67]:
sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm01041983*",
    security_token="dxbNevVOmgmoQS4Zllh3DpNDj",
)

In [68]:
results = sf.query_all(
        """
        SELECT CALENDAR_YEAR(CreatedDate), COUNT(Id),CALENDAR_MONTH(CreatedDate)
    from Paciente_Programas__c
    Where Programas_por_Empresas__r.Holding__c like '%antofagasta minerals%'
    GROUP BY CALENDAR_YEAR(CreatedDate), CALENDAR_MONTH(CreatedDate)
    ORDER BY CALENDAR_MONTH(CreatedDate)
        """
)
derttyuiop = pd.DataFrame(results["records"])
derttyuiop.to_excel("F:/Empresas SF/reportes/aaaaaa.xlsx", index=False)

In [69]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c,RUT_Empresa__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
        "RUT_Empresa__c",
    ]
]
Empresas_SF.shape


(7759, 8)

In [70]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape


(7449, 5)

In [71]:
# Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "RUT_Empresa__c": "Rut Empresa",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Rut Empresa",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A",
    ]
]

Programas_empresas.shape

(7765, 10)

In [72]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx", index=False)

PermissionError: [Errno 13] Permission denied: 'F:/Empresas SF/reportes/empresas.xlsx'

fin empresas y comienzo de rescate de pacientes


In [73]:
# CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        "Phone",
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape


(1820648, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.


In [74]:
# buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña",
    }
)
Paciente_Programas_SF.shape


(1435324, 4)

In [75]:
# Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c",
)
empresas_unidas.shape

(1435324, 13)

In [76]:
empresas_unidas.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,None,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,None,EMP6274,A-0353


In [77]:
# pacientes_empresas22 = pd.merge(
# left=Pacientes_SF,
# right=empresas_unidas,
# how="left",
# left_on="Id Account Paciente",
# right_on="Id Account",
# )
# pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
# pacientes_empresas22 = pacientes_empresas22[
# ["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
# ]

In [78]:
# pacientes_empresas22.shape

In [79]:
# apprix_1 = pacientes_empresas22.iloc[:848826,:]

# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)

In [80]:
# apprix_2 = pacientes_empresas22.iloc[848827:,:]#
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [81]:
# pacientes_empresas22.head()

Carga de Archivo carga


In [191]:
nuevos = "F:/Empresas SF/reportes/carga.xlsx"
campaña = "BCI"
Pais = "Chile"
# Pais = "México"

Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion

In [192]:
Pacientes_Archivo.head(10)


,HOLDING,poliza,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,Unnamed: 5,RUT_ASEGURADO,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,apellidos,nombres,RELACIÓN
0,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,15782029,K,15782029-K,BRUNA ROSARIO,BRUNA,ROSARIO,Titular Femenino
1,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,25426579,9,25426579-9,PIZARRO BRUNA HORACIO,PIZARRO,BRUNA HORACIO,Hijo
2,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,24658825,2,24658825-2,PIZARRO BRUNA LUCIA,PIZARRO,BRUNA LUCIA,Hija
3,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,26823567,NaN,26823567,1,26823567-1,FRANCESCHI CROQUER GERLI,FRANCESCHI,CROQUER GERLI,Titular Femenino
4,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,17339712,NaN,17339712,7,17339712-7,GUAJARDO REYES JONATHAN ALEXANDER,GUAJARDO,REYES JONATHAN ALEXANDER,Titular Masculino
5,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,8304341,NaN,8304341,5,8304341-5,HIZAUT BOLBARAN DAVID HERNAN,HIZAUT,BOLBARAN DAVID HERNAN,Titular Masculino
6,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,8304341,NaN,20297497,K,20297497-K,HIZAUT SALAZAR AMEERAH PAMELA,HIZAUT,SALAZAR AMEERAH PAMELA,Hija
7,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,8304341,NaN,11549979,3,11549979-3,SALAZAR VELASQUEZ SOLANGE ANDREA,SALAZAR,VELASQUEZ SOLANGE ANDREA,Esposa
8,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,17405936,NaN,17405936,5,17405936-5,JIMENO ORTEGA PAZ BEATRIZ,JIMENO,ORTEGA PAZ BEATRIZ,Titular Femenino
9,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,16354946,NaN,16354946,8,16354946-8,PINOCHET CARVAJAL ALEJANDRO,PINOCHET,CARVAJAL ALEJANDRO,Titular Masculino


In [193]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].astype(str)

In [194]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace("-", "")

Pacientes_Archivo.head()


,HOLDING,poliza,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,Unnamed: 5,RUT_ASEGURADO,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,apellidos,nombres,RELACIÓN,rut
0,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,15782029,K,15782029-K,BRUNA ROSARIO,BRUNA,ROSARIO,Titular Femenino,15782029K
1,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,25426579,9,25426579-9,PIZARRO BRUNA HORACIO,PIZARRO,BRUNA HORACIO,Hijo,254265799
2,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,24658825,2,24658825-2,PIZARRO BRUNA LUCIA,PIZARRO,BRUNA LUCIA,Hija,246588252
3,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,26823567,NaN,26823567,1,26823567-1,FRANCESCHI CROQUER GERLI,FRANCESCHI,CROQUER GERLI,Titular Femenino,268235671
4,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,17339712,NaN,17339712,7,17339712-7,GUAJARDO REYES JONATHAN ALEXANDER,GUAJARDO,REYES JONATHAN ALEXANDER,Titular Masculino,173397127


In [195]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()

Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_27136\3578335771.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,HOLDING,poliza,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,Unnamed: 5,RUT_ASEGURADO,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,apellidos,nombres,RELACIÓN,rut
0,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,15782029,K,15782029-K,BRUNA ROSARIO,BRUNA,ROSARIO,Titular Femenino,15782029K
1,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,25426579,9,25426579-9,PIZARRO BRUNA HORACIO,PIZARRO,BRUNA HORACIO,Hijo,254265799
2,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,24658825,2,24658825-2,PIZARRO BRUNA LUCIA,PIZARRO,BRUNA LUCIA,Hija,246588252
3,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,26823567,NaN,26823567,1,26823567-1,FRANCESCHI CROQUER GERLI,FRANCESCHI,CROQUER GERLI,Titular Femenino,268235671
4,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,17339712,NaN,17339712,7,17339712-7,GUAJARDO REYES JONATHAN ALEXANDER,GUAJARDO,REYES JONATHAN ALEXANDER,Titular Masculino,173397127


In [196]:
if ("nombres" in Pacientes_Archivo.columns) == True:
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
    Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
    # Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
    Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()

Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace("-", "")


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_27136\2830316430.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_27136\2830316430.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [197]:
# aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(29023, 24)

In [198]:
Pacientes_con_id.head()


,HOLDING,poliza,EMPLEADOR,NOMBRE_GRUPO,RUT_TITULAR,Unnamed: 5,RUT_ASEGURADO,DV_ASEGURADO,rut1,NOMBRE DEL ASEGURADO,...,RutEstaoNo,Id Account Paciente,Rut,Nombre,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,15782029,K,15782029-K,BRUNA ROSARIO,...,True,0018c00002bJxhrAAC,15782029K,Bruna Oportus Rosario,RUT,None,None,None,Mujer,None
1,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,25426579,9,25426579-9,PIZARRO BRUNA HORACIO,...,True,0018c00002bKPY6AAO,254265799,Pizarro Bruna Horacio,RUT,None,None,None,Hombre,None
2,YARUR,739012,FUNDACION YA .,FUNDACION YA,15782029,NaN,24658825,2,24658825-2,PIZARRO BRUNA LUCIA,...,True,0018c00002bKPY5AAO,246588252,Pizarro Bruna Lucia,RUT,None,None,None,Mujer,None
3,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,26823567,NaN,26823567,1,26823567-1,FRANCESCHI CROQUER GERLI,...,True,0018c00002f4SCwAAM,268235671,Gerli Franceschi Croquer,RUT,None,None,None,None,None
4,YARUR,739012,EMPRESAS JY S.A.,EMPRESAS JY S.A.,17339712,NaN,17339712,7,17339712-7,GUAJARDO REYES JONATHAN ALEXANDER,...,True,0018c00002f4SCxAAM,173397127,Jonathan Alexander Guajardo Reyes,RUT,None,None,None,None,None


In [199]:
regex = re.compile(r"([A-Za-z0-9]+[.-_])*[A-Za-z0-9]+@[A-Za-z0-9-]+(\.[A-Z|a-z]{2,})+")
def isValid(email):
    if re.fullmatch(regex, email):
        return email
    else:
        return ""

In [200]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = (
        Pacientes_con_id["rut"].astype(str).str.upper()
    )
    Subir_pacientes.head(20)
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (Pacientes_con_id["email"].astype(str).str.lower())
    Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].apply(lambda y: isValid(y))
    #Subir_pacientes["PersonEmail"] = Subir_pacientes["PersonEmail"].str.replace("no informado", "")
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]

if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        "Varón": "Hombre",
        "varon":"Hombre"
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.lower().replace("nan", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = (
        Pacientes_con_id["celular"].apply(str).str.strip().replace("+", "")
    )
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("NaN", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("Nan", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("-", "")
    #Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("0", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("No Informado", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("_", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("+", "")
    Subir_pacientes["Phone"] = Subir_pacientes["Phone"].str.replace("nan", "")

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"
Subir_pacientes["Cuenta_validada__c"] = "True"


if Pais == "Chile":
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtnEAE"
else:
    Subir_pacientes["BillingCountry"] = "México"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes["Pa_s_del_Paciente__c"] = "a1U8c000005dAtsEAE"


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,RecordTypeId,Cuenta_validada__c,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c
20,False,Cañon Alicia Del Carmen,Gatica,120666169,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
21,False,Gatica Vania Roco,Ubilla,208332155,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
344,False,Nunez Jorlen Jose,Hernandez,257543781,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
345,False,Roa Sofia,Hernandez,258393821,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE
346,False,Guanipa Yeraldin,Roa,257543803,0128c000002Byp4AAC,True,Chile,RUT,a1U8c000005dAtnEAE


In [201]:
clinica_virtual = pd.DataFrame()
clinica_virtual["rut"] = Pacientes_con_id["rut1"].str.upper().str.strip()
clinica_virtual["firstName"] = Pacientes_con_id["nombres"].str.title().str.strip()
clinica_virtual["lastName"] = Pacientes_con_id["apellidos"].str.title().str.strip()
clinica_virtual["accountId"] = campaña
""" cha_arc = datetime.now()
dia_hoy = str(fecha_arc.day)
mes_hoy = str(fecha_arc.month)
 """


Codigos_Clinica_Virtual = {
    "BCI": "1",
    "Farmacias Ahumada": "2",
    "MetLife": "3",
    "Vida Security": "4",
    "Willis": "5",
    "Entel": "6",
    "Testing": "7",
    "3SP": "8",
    "HomeCare": "9",
    "Pedro de Valdivia": "10",
    "Colaboradores": "11",
    "MineraAbra": "12",
    "CMPC": "13",
    "Lipigas": "14",
    "Asismed": "15",
    "Siempre Perú": "16",
    "Farmacias Cruz Verde": "17",
    "WFS Food Services": "18",
    "VidaCel": "19",
    "Dermik": "20",
    "EY": "21",
    "IND": "22",
    "Sindicato Walmart": "23",
    "Instruvalve": "24",
    "VidaCel Colaboradores": "25",
    "Socofar": "26",
    "Milan Fajanovic": "27",
    "THBseguros": "28",
    "Pesco": "29",
    "Burgos - Consorcio": "30",
    "COFAMA-SILENTIUM": "31",
    "KAUFMANN": "32",
    "QuePlan": "33",
    "Zurich": "34",
    "Colaboradores Siempre Chile": "35",
    "Talo": "36",
    "Transvip": "37",
    "Vida Security Colaboradores": "38",
    "FORUS": "39",
    "Sura": "40",
    "Southbrige": "41",
    "Construmart": "42",
    "BICE Vida": "43",
    "Lockton": "44",
    "Mercer": "48",
    "Grupo PSG": "46",
    "Siempre Chile": "47",
}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
# clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)
""" clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)
 """

' clinica_virtual.to_excel("F:/Cargas Base Datos CRM/Altas CV/Sin Cargar/Clinica_Virtual_"+campaña+"_"+dia_hoy+"_"+mes_hoy+".xlsx", index=False)\n '

In [202]:
""" print(dia_hoy) """


' print(dia_hoy) '

In [203]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Rut Empresa,Codigo EMP,Codigo A
157325,a1Y8c00000C6V01EAF,0018c00002amPZPAA2,a1W8c0000083rhoEAA,BCI,0018c00002axQgtAAE,Trafigura Chile Ltda.,Trafigura,BCI,Trafigura Chile Ltda.,209903,None,EMP6118,A-9328
695982,a1Y8c00000CyLq0EAF,0018c00002mSmxuAAC,a1W8c0000083rhzEAA,BCI,0018c00002axQgmAAE,Corporación Nacional Forestal,Conaf,BCI,Corporación Nacional Forestal,1298618,None,EMP6259,A-9339
698216,a1Y8c00000CySbREAV,0018c00002mTrrxAAC,a1W8c0000083rhsEAA,BCI,0018c00002axQh1AAE,Soc. Agrícola El Porvenir S.A.,Porvenir,BCI,Soc. Agrícola El Porvenir S.A.,241535,None,EMP6126,A-9332
698217,a1Y8c00000CySbSEAV,0018c00002mTrryAAC,a1W8c0000083rhsEAA,BCI,0018c00002axQh1AAE,Soc. Agrícola El Porvenir S.A.,Porvenir,BCI,Soc. Agrícola El Porvenir S.A.,241535,None,EMP6126,A-9332
698218,a1Y8c00000CySbTEAV,0018c00002mTrrzAAC,a1W8c0000083rhsEAA,BCI,0018c00002axQh1AAE,Soc. Agrícola El Porvenir S.A.,Porvenir,BCI,Soc. Agrícola El Porvenir S.A.,241535,None,EMP6126,A-9332


In [204]:
campaña_seleccionada.shape


(35263, 13)

In [205]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )


In [206]:
if campaña != "MetLife1":
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    if ("poliza" in campaña_en_pacientes.columns) == True:
        campaña_en_pacientes["poliza"] = campaña_en_pacientes["poliza"].astype(str)

    campaña_en_pacientes = campaña_en_pacientes.rename(
        columns={"Programas_por_Empresas__c": "programas por empresa"}
    )
    campaña_en_pacientes["Paciente__c"] = campaña_en_pacientes["Id Account Paciente"]
    campaña_en_pacientes["Programas_por_Empresas__c"] = campaña_en_pacientes[
        "programas por empresa"
    ]
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña2.xlsx", index=False
    )


In [207]:
# apprix_1 = Pacientes_SF.iloc[:790552,:]
# apprix_2 = Pacientes_SF.iloc[790553:,:]
# apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
# apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)

In [208]:
campaña_en_pacientes2 = pd.merge(
    left=Pacientes_con_id,
    right=campaña_seleccionada,
    how="left",
    left_on="Id Account Paciente",
    right_on="Id Account",
)
if ("poliza" in campaña_en_pacientes2.columns) == True:
    campaña_en_pacientes2["poliza"] = campaña_en_pacientes2["poliza"].astype(str)

campaña_en_pacientes2 = campaña_en_pacientes2.rename(
    columns={"Programas_por_Empresas__c": "programas por empresa"}
)
campaña_en_pacientes2["Paciente__c"] = campaña_en_pacientes2["Id Account Paciente"]
campaña_en_pacientes2["Programas_por_Empresas__c"] = campaña_en_pacientes2[
    "programas por empresa"
]


""" campaña_en_pacientes2 = campaña_en_pacientes2[
    (campaña_en_pacientes2["Programas_por_Empresas__c"] == "")] """
campaña_en_pacientes2["Esta"] = campaña_en_pacientes2["programas por empresa"].isnull()
campaña_en_pacientes2 = campaña_en_pacientes2[campaña_en_pacientes2["Esta"]]
campaña_en_pacientes2.to_excel(
    "F:/Empresas SF/reportes/archivo_campaña3.xlsx", index=False
)
